# Student Loan Risk with Deep Learning

In [46]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [47]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [48]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [49]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [50]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [51]:
# Define the target set `y` using the "credit_ranking" column
y = loans_df["credit_ranking"]

# Display a sample of the target set
y.head()


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [52]:
# Define the features set `X` by selecting all columns except "credit_ranking"
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [53]:
# Split the data into training and testing sets
# Assign random_state=1 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (1279, 11)
X_test shape: (320, 11)
y_train shape: (1279,)
y_test shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [54]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled X_train sample:\n{X_train_scaled[:5]}")
print(f"Scaled X_test sample:\n{X_test_scaled[:5]}")


Scaled X_train sample:
[[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]
 [-0.62052788  0.49684658 -1.05484864 -0.0345135  -0.20139592  0.11885252
   1.18937522  0.37546769  1.24352773 -0.76640023 -0.6873351 ]
 [-0.50797663  0.60887739 -1.00429969 -0.53491726  0.26101766 -0.74240736
  -0.7077067  -0.33312578 -0.09557544 -0.47015208 -0.77978452]]
Scaled X_test sample:
[[ 0.27988214 -0.67947695  1.87699031 -0.24897225  0.11388606 -0.64671182
  -0.11681233  0.96943574  1.43482818  0.00384497  0.05226031]
 [ 0.22360652  0.55286199  0.05722821  0.10845901  0.17694246  0.11885252
   0.72287966  0.31294474 -0.35064271 -0.17390392 -0.22508

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [55]:
# Define the number of input features for the model
number_of_features = X_train_scaled.shape[1]

# Review the number of features
print(f"Number of input features: {number_of_features}")


Number of input features: 11


In [56]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = int((number_of_features + 1) / 2) 

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = int(hidden_nodes_layer1 / 2)  

# Define the number of neurons in the output layer
output_neurons = len(y.unique())  

print(f"Input Features: {number_of_features}")
print(f"Hidden Layer 1 Nodes: {hidden_nodes_layer1}")
print(f"Hidden Layer 2 Nodes: {hidden_nodes_layer2}")
print(f"Output Layer Neurons: {output_neurons}")


Input Features: 11
Hidden Layer 1 Nodes: 6
Hidden Layer 2 Nodes: 3
Output Layer Neurons: 2


In [57]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, input_dim=number_of_features, activation='relu'))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='softmax'))  



c:\Users\Augus\anaconda3\envs\py310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
# Display the Sequential model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │             8 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [59]:
# Compile the Sequential model
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam',            
    metrics=['accuracy']
)



In [60]:

model = Sequential()
model.add(Dense(units=hidden_nodes_layer1, input_dim=number_of_features, activation='relu'))
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [61]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)

# Display the model's loss and accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4573 - loss: 0.7268
Test Loss: 0.7184
Test Accuracy: 0.4719


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [62]:
# Set the model's file path
model_file_path = Path("student_loans.keras")

# Export the model to a keras file
model.save(model_file_path)


print(f"Model saved to {model_file_path}")



Model saved to student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [63]:
# Set the model's file path
model_file_path = Path("student_loans.keras")

# Load the model into a new object
loaded_model = load_model(model_file_path)


print("Model loaded successfully!")


Model loaded successfully!


c:\Users\Augus\anaconda3\envs\py310\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [64]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Convert predictions to binary results 
predictions_binary = (predictions > 0.5).astype(int)
# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions_binary, columns=["Predictions"])

# Display a sample of the predictions
print(predictions_df.head())


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
   Predictions
0            0
1            1
2            1
3            0
4            0


In [65]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions_binary, columns=["Predictions"])

# Display a sample of the predictions
print(predictions_df.head())

   Predictions
0            0
1            1
2            1
3            0
4            0


### Step 4: Display a classification report with the y test data and predictions

In [66]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, predictions_binary, target_names=["Class 0", "Class 1"])

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.45      0.45      0.45       154
     Class 1       0.49      0.49      0.49       166

    accuracy                           0.47       320
   macro avg       0.47      0.47      0.47       320
weighted avg       0.47      0.47      0.47       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate. 
Student Profile: Age, income, employment, and credit score to assess financial capacity.
Loan Preferences: Loan amount, repayment period, and interest type to match with suitable loan products.
Educational Details: Tuition costs, institution, and graduation date to determine loan needs.
This data ensures personalized and relevant loan recommendations based on eligibility and preferences.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.
The system would use content-based filtering. Matching the student’s financial and educational profile with loan features. Content-based filtering is appropriate because it uses the collected data to recommend loans that align with the student’s unique requirements.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.
Data Privacy: Collecting sensitive financial and personal data requires strong security measures and compliance with legal standards. Poor data protection could result in breaches and loss of trust.
Bias: The system could unintentionally favor certain loans or demographics, leading to unfair recommendations. Ensuring fairness and unbiased algorithms is critical for credibility and inclusivity.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

